In [24]:
import imaplib
import email
import re

# create an IMAP client instance
mail = imaplib.IMAP4_SSL('imap.gmail.com')

# specify the keyword to search for
email_address = input('what is your email address?:\n ')

password = input('what is your App password? (This is a special password under security under settings):\n ')

keyword_to_search = input('what keyword do you want to search?:\n ')

# log in to the Gmail account
mail.login(email_address, password)

# select the inbox folder
mail.select('inbox')

# get the latest 150 email messages from the inbox
typ, data = mail.search(None, 'ALL')
latest_email_ids = data[0].split()[-500:]

print(f'Searching for keyword "{keyword_to_search}" in the body of the latest 500 email messages...')


count_keyword = 0
sender_set = set() # create an empty set to store unique email addresses of senders

# iterate over the latest 150 email messages
for email_id in latest_email_ids:
    typ, msg_data = mail.fetch(email_id, '(RFC822)')
    for response_part in msg_data:
        if isinstance(response_part, tuple):
            # parse the email message
            msg = email.message_from_bytes(response_part[1])
            
            # get the sender of the email
            sender = msg['From']
            
            # extract the body of the email as text
            if msg.is_multipart():
                for part in msg.walk():
                    if part.get_content_type() == 'text/plain':
                        try:
                            body = part.get_payload(decode=True).decode('utf-8')
                        except UnicodeDecodeError:
                            body = part.get_payload(decode=True).decode('iso-8859-1')

                        count_keyword += body.count(keyword_to_search)

                        # add the email address to the set if the keyword is found
                        if keyword_to_search.lower() in body.lower():
                            email_address = re.search(r'[\w\.-]+@[\w\.-]+', sender)
                            sender_set.add(email_address.group(0))
                            
                    elif part.get_content_maintype() == 'text':
                        try:
                            body = part.get_payload(decode=True).decode('utf-8')
                        except UnicodeDecodeError:
                            body = part.get_payload(decode=True).decode('iso-8859-1')

                        count_keyword += body.count(keyword_to_search)

                        # add the email address to the set if the keyword is found
                        if keyword_to_search.lower() in body.lower():
                            email_address = re.search(r'[\w\.-]+@[\w\.-]+', sender)
                            sender_set.add(email_address.group(0))
            else:
                try:
                    body = msg.get_payload(decode=True).decode('utf-8')
                except UnicodeDecodeError:
                    body = msg.get_payload(decode=True).decode('iso-8859-1')

                count_keyword += body.count(keyword_to_search)

                # add the email address to the set if the keyword is found
                if keyword_to_search.lower() in body.lower():
                    email_address = re.search(r'[\w\.-]+@[\w\.-]+', sender)
                    sender_set.add(email_address.group(0))

# print the total number of occurrences of the keyword
print(f'The keyword "{keyword_to_search}" appears {count_keyword} time(s) in the body of the latest 500 email messages.\n')

print('Unique sender email addresses of emails containing the keyword:\n')

print(sender_set)

what is your email address?:
 5291ccc@gmail.com
what is your App password? (This is a special password under security under settings):
 ebbolddhrxafcwtj
what keyword do you want to search?:
 review
Searching for keyword "review" in the body of the latest 500 email messages...
The keyword "review" appears 804 time(s) in the body of the latest 500 email messages.

Unique sender email addresses of emails containing the keyword:

{'donotreply@amazon.com', 'no-reply@em-s.dropbox.com', 'PayPal@emails.paypal.com', 'donotreply@member.sonyrewards.com', 'yourprovider@simplepractice.com', 'service@mail5.zocdoc.com', 'email@email.etsy.com', 'info@angloville.com', 'hello@newsletter.life360.com', 'noreply@findheadway.com', 'paypal@mail.paypal.com', 'NJFishandWildlife@public.govdelivery.com', 'zainom@mail.montclair.edu', 'prime@amazon.com', 'shipment-tracking@amazon.com', 'hello@junglescoutmail.com', 'location-history-noreply@google.com', 'marketplace-messages@amazon.com', 'noreply@robinhood.com', 'i